# Tutorial - QFrame

## What is a QFrame?
QFrame is a class which generates an SQL statement. It stores fields info in `QFrame.data` parameter which is a dictionary.

`QFrame.data` has `select` key in which it stores `fields` which we want to have in our SQL statement. Each key have to have specified `type` which can be 'dim' if the varibale is a dimension variable or 'num' if the variable is a numeric variable. Let's take a look at all options that we can have under `select` and `fields` keys.

```json
{
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "column": {
        "type": "dim",
        "as": "",
        "group_by": "",
        "order_by": "",
        "expression": "",
        "select": "",
        "custom_type": ""
      }
    },
    "where": "",
    "distinct": "",
    "having": "",
    "limit": ""
  }
}
```

- `table` - Name of the table.
- `schema` - Name of the schema.
- `fields`, in each field:
    - `type` - Type of the column. Options:

        - 'dim' - VARCHAR(500)  
        - 'num' - FLOAT
     
     Every column has to have specified type. If you want to sepcify another type check `custom_type`.
    - `as` - Column alias (name).

    - `group_by` - Aggregation type. Possibilities:

        - 'group' - This field will go to GROUP BY statement.
        - {'sum', 'count', 'min', 'max', 'avg'} - This field will by aggregated in specified way.
  
     If you don't want to aggregate fields leave `group_by` empty in each field.
    - `order_by` - Put the field in order by statement. Options:
    
        - 'ASC'
        - 'DESC'
        
    - `expression` - Expression, eg. CASE statement, column operation, CONCAT statement, ... .
    - `select` - Set 0 if you don't want to put this field in SELECT statement.
    - `custom_type` - Specify custom SQL data type, eg. DATE.
- `where` - Add where statement, eg. 'sales>100'
- `distinct` - Set 1 to add distinct to select
- `having` - Add having statement, eg. 'sum(sales)>100'
- `limit` - Add limit, eg. 100

## How to create a QFrame?
You can create a QFrame manually - passing the data directly to QFrame or automatically - using `initiate` function.

In [1]:
from grizly import (
    get_path, 
    QFrame
)

### Manually - using dictionary

This method is the most direct method of creating a QFrame - to use it you need to know the structure of `QFrame.data`. From following dictionary

In [2]:
data = {
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "col": {
        "type": "dim"
      }
    }
  }
}

QFrame will generate a simple sql

In [3]:
qf = QFrame().read_dict(data)
print(qf)

SELECT col
FROM schema.table


Here we also used simple method `.get_sql()` which prints sql saved in QFrame.

### Manually - using JSON file

We use a `.json` file to conviniently manipulate information about columns, renames and other things that might be very verbose to manipulate in python code. We can edit the json file into a json editor like http://jsoneditoronline.org/ more conviniently than in Python code.

After editing the `store.json` we can read it back inside a QFrame using `read_json()`.

This means we can use our json as our main `store` of verbose information and python as our main way to manipulate said information.

In [4]:
json_path = get_path("dev", "grizly", "notebooks", "store.json")
qf.save_json(json_path=json_path, subquery="my_query_1")

qf = QFrame().read_json(json_path=json_path, subquery="my_query_1")
print(qf)

Data saved in C:\Users\TE386850\dev\grizly\notebooks\store.json
SELECT col
FROM schema.table


### Automatically - using initiate funtion

The other way to generate a QFrame is to use `initiate` function. You can use it in two ways. First is to pass the column names directly. 

In [5]:
from grizly import initiate

initiate(columns=["col1", "col2"], 
         schema="schema", 
         table="table", 
         json_path=json_path,
         subquery="my_query_2")

qf = QFrame().read_json(json_path=json_path, subquery="my_query_2")
print(qf)

Data saved in C:\Users\TE386850\dev\grizly\notebooks\store.json
SELECT col1,
       col2
FROM schema.table


The second way is to use `get_columns` function which will import all names of the columns in given table, also with the types.

In [6]:
from grizly import SQLDB

columns, col_types = SQLDB(db='redshift').get_columns(table='table_tutorial', schema='administration', column_types=True)
initiate(columns=columns,
         col_types=col_types,
         schema="administration", 
         table="table_tutorial", 
         json_path=json_path,
         subquery="my_query_3")

qf = QFrame(engine="mssql+pyodbc://redshift_acoe").read_json(json_path=json_path, subquery="my_query_3")
print(qf)

C:\Users\TE386850\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py:384: SAWarning: Exception attempting to detect unicode returns: ProgrammingError("(pyodbc.ProgrammingError) ('25P02', '[25P02] [Amazon][Amazon Redshift] (30) Error occurred while trying to execute a query: [SQLState 25P02] ERROR:  current transaction is aborted, commands ignored until end of transaction block\\n (30) (SQLExecDirectW)')")
  "detect unicode returns: %r" % de


Data saved in C:\Users\TE386850\dev\grizly\notebooks\store.json
SELECT col1,
       col2,
       col3,
       col4
FROM administration.table_tutorial


## Working with the QFrame
There is a lot of methods which you can use to edit the QFrame. You can check them in QFrame docs. In this tutorial we will only show some of them.

### Doing some basic SQL stuff
Let's now add a `where` statement, rename some fields, add calculated field and remove some fields`.

In [7]:
qf.query("col2 > 1") #<- where
qf.rename({"col1": "items", "col2": "price"})
qf.assign(calculated_field="col4*2", 
          type='num', 
          custom_type='double precision')
qf.remove(["col3", "col4"])
print(qf)

SELECT col1 AS items,
       col2 AS price,
       col4*2 AS calculated_field
FROM administration.table_tutorial
WHERE col2 > 1


:Be aware that `rename()` method doesn't change the name of the field but only the alias (final name) of the column.:

Now you can check how the data changed calling `data` attribute.

In [8]:
qf.data

{'select': {'table': 'table_tutorial',
  'schema': 'administration',
  'fields': {'col1': {'type': '',
    'as': 'items',
    'group_by': '',
    'order_by': '',
    'expression': '',
    'select': '',
    'custom_type': 'character varying'},
   'col2': {'type': '',
    'as': 'price',
    'group_by': '',
    'order_by': '',
    'expression': '',
    'select': '',
    'custom_type': 'double precision'},
   'calculated_field': {'type': 'num',
    'as': 'calculated_field',
    'group_by': '',
    'order_by': '',
    'expression': 'col4*2',
    'custom_type': 'double precision'}},
  'engine': '',
  'where': 'col2 > 1',
  'distinct': '',
  'having': '',
  'limit': '',
  'sql_blocks': {'select_names': ['col1 as items',
    'col2 as price',
    'col4*2 as calculated_field'],
   'select_aliases': ['items', 'price', 'calculated_field'],
   'group_dimensions': [],
   'group_values': [],
   'order_by': [],
   'types': ['CHARACTER VARYING', 'DOUBLE PRECISION', 'DOUBLE PRECISION']}}}

You can see that now we also have `sql_blocks` key. You can ignore it. This key is used to build SQL statement and is generated any time `get_sql()` method is called.

### Forking

Forking qframes can be important if your data workflow needs to take the same sql table and apply different transformations to it.

Sometimes we want to fork, do some transforms, then union the QFrames back together which results into an append operation on the data side.

Let's create two copies of one QFrame.

In [9]:
qf1 = qf.copy()
qf2 = qf.copy()

## Unioning data

There are two ways of unioning two QFrames - we can union by the position of the field or by the final name of the columns (that means the alias). 

In [10]:
from grizly import union

qf1.rename({"col2": "price_1", "calculated_field": "price_2"})
qf2.rename({"col2": "price_2", "calculated_field": "price_1"})

#### Union by the positon

In [11]:
uqf_pos = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='position')
print(uqf_pos)

Data unioned successfully.
SELECT col1 AS items,
       col2 AS price_1,
       col4*2 AS price_2
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS items,
       col2 AS price_2,
       col4*2 AS price_1
FROM administration.table_tutorial
WHERE col2 > 1


#### Union by the column names

In [12]:
uqf_name = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='name')
print(uqf_name)

Data unioned successfully.
SELECT col1 AS items,
       col2 AS price_1,
       col4*2 AS price_2
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS items,
       col4*2 AS price_1,
       col2 AS price_2
FROM administration.table_tutorial
WHERE col2 > 1


You can see that in this case union changes the order of the columns. 

## Joining data

In [13]:
from grizly import join

We will be using `Chinook.sqlite` to visualize data.

In [14]:
engine_string = "sqlite:///" + get_path("dev", "grizly", "tests", "Chinook.sqlite")

### Simple join

First table is `Track` table.

In [15]:
tracks = {  'select': {
                'fields': {
                    'TrackId': { 'type': 'dim'},
                    'Name': {'type': 'dim'},
                    'AlbumId': {'type': 'dim'},
                    'Composer': {'type': 'dim'},
                    'UnitPrice': {'type': 'num'}
                },
                'table': 'Track'
            }
}
tracks_qf = QFrame(engine=engine_string).read_dict(tracks)
print(tracks_qf)

SELECT TrackId,
       Name,
       AlbumId,
       Composer,
       UnitPrice
FROM Track


In [16]:
tracks_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice
1781,1782,Gonna Keep On Tryin' Till I Win Your Love,146,Barrett Strong/Norman Whitfield,0.99
365,366,Camisa Verde 2001,32,None,0.99
1062,1063,Funk Hum,84,None,0.99
478,479,Walking Contradiction,39,Billie Joe Armstrong -Words Green Day -Music,0.99
3284,3285,"Sabbath, Bloody Sabbath",256,None,0.99


The second table is `PlaylistTrack` table. 

In [17]:
playlist_track = { "select": {
                        "fields":{
                            "PlaylistId": {"type" : "dim"},
                            "TrackId": {"type" : "dim"}
                        },
                        "table" : "PlaylistTrack"
                    }
                }

playlist_track_qf = QFrame(engine=engine_string).read_dict(playlist_track)
print(playlist_track_qf)

SELECT PlaylistId,
       TrackId
FROM PlaylistTrack


In [18]:
playlist_track_qf.to_df().sample(5)

,PlaylistId,TrackId
2133,1,789
7847,8,2805
548,1,1039
4641,5,1971
4625,5,1815


Now let's join them on `TrackId`.

In [19]:
joined_qf = join([tracks_qf,playlist_track_qf], 
                 join_type="left join", 
                 on="sq1.TrackId=sq2.TrackId")

print(joined_qf)

Data joined successfully.
SELECT sq1.TrackId AS TrackId,
       sq1.Name AS Name,
       sq1.AlbumId AS AlbumId,
       sq1.Composer AS Composer,
       sq1.UnitPrice AS UnitPrice,
       sq2.PlaylistId AS PlaylistId
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId


In [20]:
joined_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice,PlaylistId
8197,3348,The Shape of Things to Come,261,None,1.99,3
8510,3445,On the Beautiful Blue Danube,311,Johann Strauss II,0.99,1
2450,985,Medo De Escuro,78,None,0.99,1
2703,1093,Quanta (Live),86,None,0.99,5
5938,2412,By-Tor And The Snow Dog,196,Geddy Lee And Alex Lifeson/Geddy Lee And Neil ...,0.99,8


As you can see in this example `UnitPrice` is taken from the first table. By default join function is taking all fields from the first QFrame, then all the fields from the second QFrame which are not in the first and so on. If you still want to keep all fields from each QFrame we have to set `unique_col=False`. We will see in the next example how does it work.

### Multiple join

Now let's use one more table to check how does multiple join look like.

In [21]:
playlists = { "select": {
                    "fields": {
                        "PlaylistId": {"type" : "dim"},
                        "Name": {"type" : "dim"}
                    },
                    "table" : "Playlist"
                }
            }

playlists_qf = QFrame(engine=engine_string).read_dict(playlists)
print(playlists_qf)

SELECT PlaylistId,
       Name
FROM Playlist


In [22]:
playlists_qf.to_df().sample(5)

,PlaylistId,Name
7,8,Music
11,12,Classical
9,10,TV Shows
8,9,Music Videos
5,6,Audiobooks


Now if we want to join `Tracks`, `PlaylistTrack` and `Playlist` tables we can use `TrackId` and `PlaylistId`. We can see that in `Tracks` and `Playlist` tables we have the same column `Name`. Let's check the option `unique_col=False` and analyse duplicated columns.

In [23]:
joined_qf = join(qframes=[tracks_qf, playlist_track_qf, playlists_qf], 
                 join_type=['left join', 'left join'], 
                 on=['sq1.TrackId=sq2.TrackId', 'sq2.PlaylistId=sq3.PlaylistId'], 
                 unique_col=False)

Data joined successfully.
Please remove or rename duplicated columns. Use your_qframe.show_duplicated_columns() to check duplicates.


In [24]:
joined_qf.show_duplicated_columns()

 DUPLICATED COLUMNS: 
 
TrackId:	 ['sq1.TrackId', 'sq2.TrackId']

Name:	 ['sq1.Name', 'sq3.Name']

PlaylistId:	 ['sq2.PlaylistId', 'sq3.PlaylistId']

Use your_qframe.remove() to remove or your_qframe.rename() to rename columns.


We can see that three columns occure in two different tables. We will remove `sq2.TrackId` and  `sq2.PlaylistId` fields and rename `Name` column.

In [25]:
joined_qf.remove(['sq2.TrackId', 
                  'sq2.PlaylistId']).rename({'sq1.Name': 'TrackName', 
                                             'sq3.Name': 'PlaylistType'})
print(joined_qf)

SELECT sq1.TrackId AS TrackId,
       sq1.Name AS TrackName,
       sq1.AlbumId AS AlbumId,
       sq1.Composer AS Composer,
       sq1.UnitPrice AS UnitPrice,
       sq3.PlaylistId AS PlaylistId,
       sq3.Name AS PlaylistType
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId
LEFT JOIN
  (SELECT PlaylistId,
          Name
   FROM Playlist) sq3 ON sq2.PlaylistId=sq3.PlaylistId


In [26]:
joined_qf.to_df().sample(5)

,TrackId,TrackName,AlbumId,Composer,UnitPrice,PlaylistId,PlaylistType
7720,3139,Slow An' Easy,141,Moody,0.99,1,Music
6907,2795,Bichos Escrotos (Vinheta),224,Titãs,0.99,5,90’s Music
5626,2292,Turn You Inside-Out,188,Bill Berry-Peter Buck-Mike Mills-Michael Stipe,0.99,1,Music
7515,3050,Superstition,241,None,0.99,1,Music
318,125,"Spanish moss-""A sound portrait""-Spanish moss",13,Billy Cobham,0.99,8,Music
